In [5]:
from PdfCleaner import PdfCleaner
from PerformanceTesting import PerformanceTester
from os import listdir
from os.path import isfile, join
import pdfplumber
import json
import pandas as pd
import numpy as np
import requests
from pymongo import MongoClient



class Transcripts:
    def __init__(self, ticker):
        self.ticker = ticker
        xinyu = "/Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project"
        marco = "C:/Users/mtort/Repositories/DS3500-Final-Project"
        emily = "/Users/emilywang/Desktop/DS3500-Final-Project-main-2"
        kelly = "/Users/kelly/Desktop/ds3500/DS3500-Final-Project"
        qi = ""
        self.path = xinyu +"/transcripts/"+ticker+"_transcripts/"

    def read_files(self):
        return [f for f in listdir(self.path) if isfile(join(self.path, f))]

    def create_dct(self):
        lst_files = self.read_files()
        lst_cleaned = []
        for file in lst_files:
            file_path = self.path + file
            if file_path[-3:] == 'pdf':
                txt = PdfCleaner(file_path)
                date = file[:8]
                # txt_cleaned = txt.clean_stopwords_punctuation()
                txt_cleaned = txt.clean_nums()
                PerfTest = PerformanceTester()
                PerfTest.setTimeframe('day', 1)
                PerfTest.loadArticles([[self.ticker, date, txt_cleaned]])
                try:
                    classification_xy = PerfTest.aquireTargetValues()
                except KeyError:
                    print(f'''Warning: attempted to access market during weekend or after hours
                          Earnings Transcript {file_path} Not Added
                          ''')
                    continue
                dct_cleaned = {'price_change': classification_xy[1][0], 'name': self.ticker, 'date': date, 'transcript': txt_cleaned}
                lst_cleaned.append(dct_cleaned)
        return lst_cleaned

    
class Database:
    def __init__(self):
        client = MongoClient()
        client.drop_database('transcripts')
        self.db = client.transcripts

    def store_data(self, tickers_lst):
        for t in tickers_lst:
            store = Transcripts(t)
            transcript = store.create_dct()
            self.db.transcript.insert_many(transcript)
            print(t + " transcripts stored successfully")
        return self.db

# def main():

    


# if __name__ == '__main__':
#     main()

In [8]:
tickers = ['AAPL']
    # works: APPL, MSFT
    # doesn't work: FB, GOOGL, NFLX, TSLA, ADBE, CMCSA, COST, AMZN

    # 'APPL', 'MSFT', 'FB', 'GOOGL', 'NFLX', 'TSLA', 'ADBE', 'CMCSA', 'COST', 'AMZN'
    # client = MongoClient()
    # client.drop_database('transcripts')
    # db = client.transcripts


    # for t in tickers:
    #     store = Transcripts(t)
    #     transcript = store.create_dct()
    #     # print(transcript)
    #     db.transcript.insert_many(transcript)
    #     print(t + " transcripts stored successfully")

    # db.transcript.find() # client.transcripts.transcript.find()

database = Database()
db = database.store_data(tickers)
all_transcripts = db.transcript.find()
for transcript in all_transcripts:
    print(transcript)

                          Earnings Transcript /Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526178.pdf Not Added
                          
                          Earnings Transcript /Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526172.pdf Not Added
                          
AAPL transcripts stored successfully
{'_id': ObjectId('61a6912f6846cbd1574bcd2c'), 'price_change': 0, 'name': 'AAPL', 'date': '20170801', 'transcript': 'Company Name Apple Inc Company Ticker AAPL US Equity Date  Q  Earnings Call Company Participants Luca Maestri Chief Financial Oﬃcer  Senior Vice President L Nancy Paxton Senior Director Investor Relations Treasury A N Timothy Donald Cook Chief Executive Oﬃcer  Director I F Other Participants Amit Daryanani Analyst Antonio M Sacconaghi Analyst Brian J White Analyst 

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


''' Query to test on all APPLE transcripts '''
# apple = db.transcript.find({"name":"APPL"})


''' Query to test on all transcripts '''
all_companies = db.transcript.find()

data = {}
text = []

for transcript in all_companies:
    data[transcript["name"]+" "+transcript["date"]] = transcript["transcript"]
    text.append(transcript["transcript"])
#     print(transcript)
#     df = pd.DataFrame.from_dict(data, orient ='index')




# for name, info in data.items():
''' Use countvectorizer to convert pdf text to a matrix of token counts '''
# vect = CountVectorizer()
# vect.fit(text)
#     print("vocabulary: \n", vect.vocabulary_)


''' Use bag of words to count how many times each word appears in pdf '''
# bag_of_words = vect.transform(text)
#     print(bag_of_words)

    # number of times each unique word appears in this text
# bag_of_words_arr = bag_of_words.toarray()
# #     print(bag_of_words_arr)
# feature_names = vect.get_feature_names()
#     print(feature_names)

''' Display occurrences in dataframe '''
# bow_df = pd.DataFrame(bag_of_words_arr, columns = feature_names)
# bow_df["Transcript (Company)"] = "APPL"
# bow_df.set_index("Transcript (Company & Date)", inplace=True)

''' Convert bag of word values into TFIDF values (based on the number of occurrences for each word) '''

vect = TfidfVectorizer(min_df=3, ngram_range = (1, 1)).fit(text)
bag_of_words = vect.transform(text)
feature_names = vect.get_feature_names()
    
tfidf_df = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
# tfidf_df[name] = text
# tfidf_df.set_index(name, inplace=True)

# tfidf_df.head()
    
    
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(bag_of_words_arr).toarray()

    
# tfidf_df = pd.DataFrame(X, columns = feature_names)
# tfidf_df["Transcript (Company)"] = "APPL"
# tfidf_df.set_index("Transcript (Company & Date)", inplace=True)

# ''' Show number of occurences of each word in one dataframe and tfidf (weighted) values of each word in another
#     dataframe for each pdf '''
# display(bow_df)
display(tfidf_df)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [35]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

''' Using PCA to reduce total number of features before feeding into ML model '''
# instantiate the PCA object and request ten components (reduces number of columns/features)
pca = PCA(n_components=10, random_state=3000)


# standardize the features so they are all on the same scale
features_standardized = StandardScaler().fit_transform(bag_of_words.toarray())

# transform the standardized features using the PCA algorithm 
reduced_data = pca.fit_transform(features_standardized)

# show transformed results in dataframe
components = ["Component1", "Component2", "Component3", "Component4", "Component5", "Component6",
             "Component7", "Component8", "Component9", "Component10"]
reduced_df = pd.DataFrame(reduced_data, columns = components)

reduced_df



,Component1,Component2,Component3,Component4,Component5,Component6,Component7,Component8,Component9,Component10
0,31.624239,-7.203018,-1.723161,41.410040,-7.863261,26.364508,41.871851,-4.407624,10.375200,17.017196
1,25.516135,-9.458033,2.175694,2.895250,10.830417,9.582134,-1.600798,7.649630,5.004344,-7.024912
2,18.097628,15.790614,-4.287530,-14.087048,38.846732,-1.231603,10.605256,1.503559,-19.331266,-1.066348
3,18.046156,21.724962,-11.708877,8.581565,-6.848263,-9.827603,-6.708316,-8.507765,-18.682608,-2.356166
4,29.258110,-10.482088,5.750694,-12.475951,24.484326,-16.675351,-23.370572,-23.970720,30.359718,34.406944
5,17.478830,22.060806,-10.710655,9.810094,-6.600250,-11.719160,-2.376372,2.675634,-10.907716,-3.257980
6,28.216917,-18.495965,11.272190,-9.809318,5.732066,11.126531,-4.247689,-4.982623,2.645537,-2.190980
7,32.228791,-13.865795,9.302348,-13.763603,64.088783,16.873536,7.046306,-5.030649,-27.762669,-5.446354
8,28.461432,-5.112159,-1.606764,9.692875,4.327152,12.196742,-10.391975,21.934994,-10.865304,-27.138399
9,24.139252,-14.370053,9.591650,-19.965399,-13.555652,-6.222310,10.816646,1.325583,0.126071,-2.129466
